# --- Day 16: Packet Decoder ---
## Part 1
As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (BITS), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in hexadecimal (your puzzle input).

The first step of decoding the message is to convert the hexadecimal representation into binary. Each character of hexadecimal corresponds to four bits of binary data:
```
0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111
```
The BITS transmission contains a single packet at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra 0 bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet version, and the next three bits encode the packet type ID. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first. For example, a version encoded as the binary sequence 100 represents the number 4.

Packets with type ID 4 represent a literal value. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a 1 bit except the last group, which is prefixed by a 0 bit. These groups of five bits immediately follow the packet header. For example, the hexadecimal string D2FE28 becomes:
```
110100101111111000101000
VVVTTTAAAAABBBBBCCCCC
```
Below each bit is a label indicating its purpose:

- The three bits labeled V (110) are the packet version, 6.
- The three bits labeled T (100) are the packet type ID, 4, which means the packet is a literal value.
- The five bits labeled A (10111) start with a 1 (not the last group, keep reading) and contain the first four bits of the number, 0111.
- The five bits labeled B (11110) start with a 1 (not the last group, keep reading) and contain four more bits of the number, 1110.
- The five bits labeled C (00101) start with a 0 (last group, end of packet) and contain the last four bits of the number, 0101.
- The three unlabeled 0 bits at the end are extra due to the hexadecimal representation and should be ignored.

So, this packet represents a literal value with binary representation 011111100101, which is 2021 in decimal.

Every other type of packet (any packet with a type ID other than 4) represent an operator that performs some calculation on one or more sub-packets contained within. Right now, the specific operations aren't important; focus on parsing the hierarchy of sub-packets.

An operator packet contains one or more packets. To indicate which subsequent binary data represents its sub-packets, an operator packet can use one of two modes indicated by the bit immediately after the packet header; this is called the length type ID:

- If the length type ID is 0, then the next 15 bits are a number that represents the total length in bits of the sub-packets contained by this packet.
- If the length type ID is 1, then the next 11 bits are a number that represents the number of sub-packets immediately contained by this packet.

Finally, after the length type ID bit and the 15-bit or 11-bit field, the sub-packets appear.

For example, here is an operator packet (hexadecimal string 38006F45291200) with length type ID 0 that contains two sub-packets:
```
00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB
```

- The three bits labeled V (001) are the packet version, 1.
- The three bits labeled T (110) are the packet type ID, 6, which means the packet is an operator.
- The bit labeled I (0) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.
- The 15 bits labeled L (000000000011011) contain the length of the sub-packets in bits, 27.
- The 11 bits labeled A contain the first sub-packet, a literal value representing the number 10.
- The 16 bits labeled B contain the second sub-packet, a literal value representing the number 20.

After reading 11 and 16 bits of sub-packet data, the total length indicated in L (27) is reached, and so parsing of this packet stops.

As another example, here is an operator packet (hexadecimal string EE00D40C823060) with length type ID 1 that contains three sub-packets:
```
11101110000000001101010000001100100000100011000001100000
VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC
```

- The three bits labeled V (111) are the packet version, 7.
- The three bits labeled T (011) are the packet type ID, 3, which means the packet is an operator.
- The bit labeled I (1) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.
- The 11 bits labeled L (00000000011) contain the number of sub-packets, 3.
- The 11 bits labeled A contain the first sub-packet, a literal value representing the number 1.
- The 11 bits labeled B contain the second sub-packet, a literal value representing the number 2.
- The 11 bits labeled C contain the third sub-packet, a literal value representing the number 3.

After reading 3 complete sub-packets, the number of sub-packets indicated in L (3) is reached, and so parsing of this packet stops.

For now, parse the hierarchy of the packets throughout the transmission and add up all of the version numbers.

Here are a few more examples of hexadecimal-encoded transmissions:

- 8A004A801A8002F478 represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of 16.
- 620080001611562C8802118E34 represents an operator packet (version 3) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of 12.
- C0015000016115A2E0802F182340 has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of 23.
- A0016C880162017C3686B18A3D4780 is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of 31.

Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?


In [1]:
from enum import Enum
import numpy as np
from typing import Dict, List, Tuple

In [2]:
class Packet:
    class PacketType(Enum):
        LITERAL = 0
        OPERATOR = 1
    
    class InputType(Enum):
        HEXADEC = 16
        BINARY = 2
        
    __hex2bin_dict: Dict[str, bin] = {
        "0":"0000", "1":"0001", "2":"0010", "3":"0011",
        "4":"0100", "5":"0101", "6":"0110", "7":"0111",
        "8":"1000", "9":"1001", "A":"1010", "B":"1011",
        "C":"1100", "D":"1101", "E":"1110", "F":"1111"
    }
    __hex: str
    __bits: str
    __subpackets: List["Packet"]
    __length_type_id: int
    __subpackets_length: int
    __expected_n_subpackets: int
    __remainder: str
    
    __literal_bits: str
    __literal_value: int

    def __init__(self, input_string: str, input_type: InputType = InputType.HEXADEC):
        if input_type == self.InputType.HEXADEC:
            self.__hex = input_string
            self.__bits = "".join([self.__hex2bin_dict[char] for char in self.hexadec])
        else:
            self.__hex = ""
            self.__bits = input_string
        self.__subpackets = []
        if self.packet_type == self.PacketType.OPERATOR:
            self.__length_type_id = int(self.__bits[6])
            self.__exploit_length_type_id()
        else:
            self.__parse_literal_value()
    
    @property
    def hexadec(self) -> str:
        return self.__hex
    
    @property
    def bits(self) -> str:
        return self.__bits
    
    @property
    def version(self) -> int:
        return int(self.__bits[:3], 2)
    
    @property
    def packet_id(self) -> int:
        return int(self.__bits[3:6], 2)
    
    @property
    def packet_type(self) -> PacketType:
        return self.PacketType.LITERAL if self.packet_id == 4 else self.PacketType.OPERATOR
    
    @property
    def length_type_ID(self) -> Tuple[int, int]:
        if self.__length_type_id:
            return (self.__length_type_id, self.__expected_subpackets)
        else:
            return (self.__length_type_id, self.__subpacket_length)
    
    @property
    def n_subpackets(self) -> int:
        return len(self.__subpackets)
    
    @property
    def remainder(self) -> str:
        return self.__remainder
    
    @staticmethod
    def hex2bin(hex_string: str) -> str:
        return "".join([Packet.__hex2bin_dict[char] for char in hex_string])
    
    def iterative_version_sum(self) -> int:
        return self.version + sum([sp.iterative_version_sum() for sp in self.__subpackets])
    
    def reset_remainder(self) -> None:
        self.__remainder = ""
    
    def has_remainder(self) -> bool:
        return any([c != "0" for c in self.__remainder])
    
    def get_all_bits(self) -> str:
        return self.bits + "".join([sp.get_all_bits() for sp in self.__subpackets]) + (self.__remainder if not self.has_remainder() else "")
    
    def __exploit_length_type_id(self) -> None:
        if self.__length_type_id:
            self.__expected_subpackets = int(self.__bits[7:18], 2)
            self.__subpackets.append(Packet(self.__bits[18:], self.InputType.BINARY))
            while self.__subpackets[-1].has_remainder() and self.__expected_subpackets > 1:
                self.__subpackets.append(Packet(self.__subpackets[-1].remainder, self.InputType.BINARY))
                self.__subpackets[-2].reset_remainder()
                if len(self.__subpackets) == self.__expected_subpackets:
                    break
            if len(self.__subpackets) != self.__expected_subpackets:
                raise AssertionError(f"expected {self.__expected_subpackets} subpackets, found {len(self.__subpackets)} instead")
            self.__remainder = self.__subpackets[-1].remainder
            self.__subpackets[-1].reset_remainder()
            self.__bits = self.__bits[:18]
        else:
            self.__subpacket_length = int(self.__bits[7:22], 2)
            self.__subpackets.append(Packet(self.__bits[22:22+self.__subpacket_length], self.InputType.BINARY))
            while self.__subpackets[-1].has_remainder():
                self.__subpackets.append(Packet(self.__subpackets[-1].remainder, self.InputType.BINARY))
                self.__subpackets[-2].reset_remainder()
            self.__remainder = self.__bits[22+self.__subpacket_length:]
            self.__bits = self.__bits[:22]
    
    def __parse_literal_value(self) -> None:
        if not self.packet_type == self.PacketType.LITERAL:
            raise AssertionError("packet is not LITERAL")
        
        self.__literal_bits = ""
        i = 6
        for i in range(6, len(self.__bits), 5):
            self.__literal_bits += self.__bits[i+1:i+5]
            if self.__bits[i] == "0":
                break
        self.__remainder = self.__bits[(i+5):]
        self.__bits = self.__bits[:(i+5)]
        self.__literal_value = int(self.__literal_bits, 2)
    
    @property
    def operator_label(self) -> str:
        return {
            0:"SUM", 1:"PROD", 2:"MIN", 3:"MAX",
            4:"VAL", 5:"GT", 6:"LT", 7:"ET"
        }[self.packet_id]
    
    def calculate(self) -> int:
        if self.packet_id == 0:
            return sum([sp.calculate() for sp in self.__subpackets])
        elif self.packet_id == 1:
            return np.prod([sp.calculate() for sp in self.__subpackets])
        elif self.packet_id == 2:
            return np.min([sp.calculate() for sp in self.__subpackets])
        elif self.packet_id == 3:
            return np.max([sp.calculate() for sp in self.__subpackets])
        elif self.packet_id == 4:
            if self.packet_type != self.PacketType.LITERAL:
                raise AssertionError("packet ID 4 should be type LITERAL")
            return self.__literal_value
        elif self.packet_id == 5:
            if not self.n_subpackets == 2:
                raise AssertionError(f"expected 2 subpackets, {self.n_subpackets} instead.")
            return int(self.__subpackets[0].calculate() > self.__subpackets[1].calculate())
        elif self.packet_id == 6:
            if not self.n_subpackets == 2:
                raise AssertionError(f"expected 2 subpackets, {self.n_subpackets} instead.")
            return int(self.__subpackets[0].calculate() < self.__subpackets[1].calculate())
        elif self.packet_id == 7:
            if not self.n_subpackets == 2:
                raise AssertionError(f"expected 2 subpackets, {self.n_subpackets} instead.")
            return int(self.__subpackets[0].calculate() == self.__subpackets[1].calculate())
    
    def __repr__(self) -> str:
        printable = f"""|\n|--{self.bits}{":" if len(self.remainder) > 0 else ""}{self.remainder}"""
        printable += f"\n|  Version {self.version} ({self.__bits[:3]}), packet type: {self.packet_type.name} ({self.operator_label})"
        printable += f"\n|  Packet ID #{self.packet_id} ({self.__bits[3:6]}), length {len(self.__bits)}"""
        if self.packet_type == self.PacketType.LITERAL:
            printable += f"\n|  Value: {self.calculate()} ({self.__literal_bits})"
            return printable
        
        printable += f"\n|  Length type ID: {self.length_type_ID}\n|  Contains {self.n_subpackets} subpackets.\n|  Value: {self.calculate()}"""
        return printable
    
    def deep_print(self, silent: bool=False) -> str:
        printable = f"{self}\n"
        for sp in self.__subpackets:
            printable += "|  " + sp.deep_print(True).replace("\n|  ", "\n|  |  ").replace("\n|--", "\n|  |--")
        if not silent:
            print(printable)
        return printable

In [3]:
packet = Packet("D2FE28")
#s = packet.deep_print()
packet.iterative_version_sum()

6

In [4]:
packet = Packet("38006F45291200")
#s = packet.deep_print()

In [5]:
packet = Packet("EE00D40C823060")
#s = packet.deep_print()

In [6]:
packet = Packet("8A004A801A8002F478")
#s = packet.deep_print()
packet.iterative_version_sum()

16

In [7]:
packet = Packet("620080001611562C8802118E34")
#s = packet.deep_print()
packet.iterative_version_sum()

12

In [8]:
packet = Packet("C0015000016115A2E0802F182340")
#s = packet.deep_print()
packet.iterative_version_sum()

23

In [9]:
packet = Packet("A0016C880162017C3686B18A3D4780")
#s = packet.deep_print()
packet.iterative_version_sum()

31

In [10]:
with open("../data/day16.dat", "r") as IH:
    hexadec = next(IH).strip()
packet = Packet(hexadec)
#s = packet.deep_print()
packet.iterative_version_sum()

906

## --- Part Two ---

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID 4) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.

Using these rules, you can now work out the value of the outermost packet in your BITS transmission.

For example:

- C200B40A82 finds the sum of 1 and 2, resulting in the value 3.
- 04005AC33890 finds the product of 6 and 9, resulting in the value 54.
- 880086C3E88112 finds the minimum of 7, 8, and 9, resulting in the value 7.
- CE00C43D881120 finds the maximum of 7, 8, and 9, resulting in the value 9.
- D8005AC2A8F0 produces 1, because 5 is less than 15.
- F600BC2D8F produces 0, because 5 is not greater than 15.
- 9C005AC2F8F0 produces 0, because 5 is not equal to 15.
- 9C0141080250320F1802104A08 produces 1, because 1 + 3 = 2 * 2.

What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?


In [11]:
packet = Packet("C200B40A82")
#packet.deep_print()
packet.calculate()

3

In [12]:
packet = Packet("04005AC33890")
#packet.deep_print()
packet.calculate()

54

In [13]:
packet = Packet("880086C3E88112")
#packet.deep_print()
packet.calculate()

7

In [14]:
packet = Packet("CE00C43D881120")
#packet.deep_print()
packet.calculate()

9

In [15]:
packet = Packet("D8005AC2A8F0")
#packet.deep_print()
packet.calculate()

1

In [16]:
packet = Packet("F600BC2D8F")
#packet.deep_print()
packet.calculate()

0

In [17]:
packet = Packet("9C005AC2F8F0")
#packet.deep_print()
packet.calculate()

0

In [18]:
packet = Packet("9C0141080250320F1802104A08")
#packet.deep_print()
packet.calculate()

1

In [19]:
with open("../data/day16.dat", "r") as IH:
    hexadec = next(IH).strip()
packet = Packet(hexadec)
packet.calculate()

819324480368